In [69]:
import pandas as pd
import re
import nltk
import numpy as np
import string
from collections import Counter
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import seaborn as sns
import networkx as nx

display(Markdown("# Downloads Needed Dependencies and Data"))
# Download NLTK resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')



# Downloads Needed Dependencies and Data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [70]:
display(Markdown("# Reads Lscs Member Data"))
# lscs members
lscs = pd.read_csv(
    "lscs_members.csv",
    on_bad_lines="warn"
)

lscs.head()

# Reads Lscs Member Data

/tmp/ipython-input-2032914602.py:3: ParserWarning: Skipping line 743: expected 13 fields, saw 15

  lscs = pd.read_csv(


,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link
0,11825782,"Bernaldez, Leo Mikhail B.",NaN,leo_bernaldez@dlsu.edu.ph,NaN,MEM,MEM,GCOE,BS - Chemical Engineering,NaN,NaN,09052898843,https://www.facebook.com/beltran.bernaldez
1,11941820,"De Jesus, Matthew Joseph M.",NaN,matthew.dejesus@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies (CCS),BS - Computer Science Major in Network and Inf...,NaN,"Gaming, Editing",0917 618 8375,https://www.facebook.com/matti42/
2,12033391,"Palencia, Ralph Dwanne V",NaN,ralph_palencia@dlsu.edu.ph,NaN,MEM,MEM,SOE,BS Applied Industrial Economics,NaN,"Tech, games",09567349274,fb.com/dwanne.palencia
3,12038660,"Fong, Hannah Regine C.",NaN,hannah_regine_fong@dlsu.edu.ph,NaN,MEM,MEM,CCS,BSMS - Computer Science,NaN,NaN,09177012643,fb.com/fong.hannah
4,12073067,"de Ramos, Ghrazielle Rei A.",NaN,ghrazielle_deramos@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS Computer Science Major in Software Technology,NaN,"Basketball, Sketching, Coding, Volunteering",0933 862 0716,https://web.facebook.com/ghrazielle/


In [71]:
display(Markdown("# Defines House Data"))
# Define the houses
houses = {
    "Red": {"House Name": "House of Jasper", "Gem/Crystal": "Jasper", "Signature Color/Look": "earthy red/brown-red"},
    "Orange": {"House Name": "House of Carnelian", "Gem/Crystal": "Carnelian", "Signature Color/Look": "fiery orange"},
    "Yellow": {"House Name": "House of Citrine", "Gem/Crystal": "Citrine", "Signature Color/Look": "golden yellow"},
    "Green": {"House Name": "House of Aventurine", "Gem/Crystal": "Aventurine", "Signature Color/Look": "shimmering green"},
    "Light Blue": {"House Name": "House of Amazonite", "Gem/Crystal": "Amazonite", "Signature Color/Look": "turquoise blue-green"},
    "Indigo/Dark Blue": {"House Name": "House of Lapis Lazuli", "Gem/Crystal": "Lapis Lazuli", "Signature Color/Look": "deep royal blue"},
    "Purple": {"House Name": "House of Amethyst", "Gem/Crystal": "Amethyst", "Signature Color/Look": "violet to purple"},
    "White": {"House Name": "House of Moonstone", "Gem/Crystal": "Moonstone", "Signature Color/Look": "milky white with rainbow sheen"},
    "Black": {"House Name": "House of Obsidian", "Gem/Crystal": "Obsidian / Onyx", "Signature Color/Look": "jet black (volcanic glass)"},
    "Pink/Brown": {"House Name": "House of Rose Quartz", "Gem/Crystal": "Serpentine", "Signature Color/Look": "pink"}
}

# Convert to DataFrame
house_df = pd.DataFrame.from_dict(houses, orient="index").reset_index()

# Rename columns
house_df.rename(columns={"index": "Color Category"}, inplace=True)

# Add numeric ID column
house_df.insert(0, "House ID", range(len(house_df)))

house_df


# Defines House Data

,House ID,Color Category,House Name,Gem/Crystal,Signature Color/Look
0,0,Red,House of Jasper,Jasper,earthy red/brown-red
1,1,Orange,House of Carnelian,Carnelian,fiery orange
2,2,Yellow,House of Citrine,Citrine,golden yellow
3,3,Green,House of Aventurine,Aventurine,shimmering green
4,4,Light Blue,House of Amazonite,Amazonite,turquoise blue-green
5,5,Indigo/Dark Blue,House of Lapis Lazuli,Lapis Lazuli,deep royal blue
6,6,Purple,House of Amethyst,Amethyst,violet to purple
7,7,White,House of Moonstone,Moonstone,milky white with rainbow sheen
8,8,Black,House of Obsidian,Obsidian / Onyx,jet black (volcanic glass)
9,9,Pink/Brown,House of Rose Quartz,Serpentine,pink


In [72]:
display(Markdown("# Clean your 'interests' column"))

# Clean your "interests" column
lscs["cleaned_interests"] = (
    lscs["interests"]
    .astype(str)
    .replace(r"[&/;\n|*+-]", ",", regex=True)  # added + and -
    .str.replace(r",+", ",", regex=True)        # collapse multiple commas
    .str.strip(", ")                            # strip leading/trailing commas/spaces
)

initial = lscs['cleaned_interests'].sort_values()
get_nan = lscs[lscs['cleaned_interests'].str.contains('nan',case=False)]
initial

# Clean your 'interests' column

,cleaned_interests
746,1. Coding (I worked on a platform game with tw...
789,3D Modeling and Video Editing
5,?
708,"ACADEMIC,Math, Science, History, Philosophy,MU..."
777,"Anime, Deepwoken, Overwatch, Pokémon, Genshin ..."
...,...
37,what if ayaw ko
56,writing
802,"writing, drawing, tech, games, music"
456,"writing, graphic design, walking"


In [73]:
display(Markdown("# Extracts Keywords from Interests"))
# Build phrase map automatically from dataframe
stop_words = set(stopwords.words('english'))

# Collect all words from the dataframe
all_words = []
for text in lscs['cleaned_interests']:
    tokens = [w.lower() for w in word_tokenize(str(text)) if w not in string.punctuation and w not in stop_words]
    all_words.extend(tokens)

# Detect top bigrams
bigram_finder = BigramCollocationFinder.from_words(all_words)
top_bigrams = bigram_finder.nbest(BigramAssocMeasures.likelihood_ratio, 20)  # top 20 bigrams

# Detect top trigrams
trigram_finder = TrigramCollocationFinder.from_words(all_words)
top_trigrams = trigram_finder.nbest(TrigramAssocMeasures.likelihood_ratio, 10)  # top 10 trigrams

# Create phrase map: join words with _ so they become single tokens
phrase_map = {" ".join(b): "_".join(b) for b in top_bigrams + top_trigrams}

# Define keyword extraction function with nouns and verbs
def extract_keywords_auto(sentence):
    sentence_lower = str(sentence).lower()

    # Apply phrase mapping
    for phrase, replacement in phrase_map.items():
        if phrase in sentence_lower:
            sentence_lower = sentence_lower.replace(phrase, replacement)

    # Tokenize
    words = word_tokenize(sentence_lower)
    # Remove punctuation and stopwords
    words = [w for w in words if w not in string.punctuation and w not in stop_words]

    # POS tagging
    tagged = pos_tag(words)

    # Keep nouns and verbs
    keywords = [word for word, tag in tagged if tag.startswith('NN') or tag.startswith('VB')]

    # Return unique keywords
    return list(set(keywords))

# Apply to DataFrame
lscs["keywords_list"] = lscs["cleaned_interests"].apply(extract_keywords_auto)

# Save and inspect
keywords = lscs['keywords_list'].sort_values()

# Count empty keyword rows
empty_keywords = lscs['keywords_list'].apply(lambda x: len(x) == 0).sum()
print("Number of empty keyword rows: ", empty_keywords)

# Get rows with empty keywords
empty_rows = lscs[lscs['keywords_list'].apply(lambda x: len(x) == 0)]

# Display their names (or any column you want)
empty_names = empty_rows['full_name'].tolist()
print("Empty keywords found for these names:")
print(empty_names)


# Extracts Keywords from Interests

Number of empty keyword rows:  2
Empty keywords found for these names:
['Rejano, Hans Martin F.', 'Arous, Adam']


In [74]:
# Convert list of keywords into a comma-separated string
lscs["keywords"] = lscs["keywords_list"].apply(lambda x: ", ".join(x) if x else "")

# Sort the keywords column alphabetically
lscs_sorted = lscs.sort_values(by="keywords")

lscs_sorted['keywords_list']

,keywords_list
642,[]
5,[]
793,"[anime, basketball, art, learning, language, g..."
367,"[anime, calisthenics, gaming, studying, progra..."
742,"[anime, dancing, kpop, video_games, volleyball]"
...,...
821,"[years, solitaire, –_i_like, reading, –_i_like..."
628,"[years, spec, pop, articles, handled, website,..."
156,"[zone, zenless, dogs]"
768,"[👍, technology, computer]"


In [75]:
display(Markdown("# Flatten and create a DataFrame of Keywords"))
# Flatten and create a DataFrame
keywords_df = lscs['keywords_list'].explode().dropna().to_frame(name="keywords_list")

keywords_df = keywords_df.sort_values(by="keywords_list").reset_index(drop=True)
keywords_df

# Flatten and create a DataFrame of Keywords

,keywords_list
0,'m
1,'ve
2,",2."
3,",cybersecurity"
4,",video_games"
...,...
2754,’
2755,’
2756,👍
2757,👽


In [76]:
display(Markdown("# Cleans Data"))

def clean_keyword(word):
    if not isinstance(word, str):
        return None
    # Lowercase + strip spaces
    word = word.lower().strip()
    # Remove leading/trailing punctuation
    word = re.sub(r'^[\W_]+|[\W_]+$', '', word)
    # Remove if empty or just numbers/punctuation
    if word == "" or re.fullmatch(r"[\W_]+", word) or word.isdigit():
        return None
    return word

# Apply cleaning
keywords_df["keywords_list"] = keywords_df["keywords_list"].apply(clean_keyword)

# Drop None/empty
keywords_df = keywords_df.dropna().drop_duplicates().reset_index(drop=True)
keywords_df = keywords_df.dropna(subset=["keywords_list"])
keywords_df = keywords_df[~keywords_df['keywords_list'].isin(["nan"])].reset_index(drop=True)


print(keywords_df.shape[0])
print(keywords_df.head(20))


# Cleans Data

749
    keywords_list
0               m
1              ve
2   cybersecurity
3     video_games
4           acads
5          action
6      activities
7     advertising
8      aesthetics
9     aftermarket
10             ai
11            ako
12       analysis
13      analytics
14         animal
15       animanga
16      animation
17          anime
18          animo
19       annoying


In [77]:
display(Markdown("""
# 🔑 Flatten and Cluster Keywords

We are using **SentenceTransformers** to embed the keywords and then applying **KMeans clustering** with `n_clusters = 10`.

Steps performed:
1. Flatten the keywords and create a DataFrame.
2. Generate embeddings using `all-mpnet-base-v2`.
3. Apply KMeans clustering on embeddings.
4. Assign cluster labels to each keyword.
5. Display sorted keywords and cluster counts.
"""))

model = SentenceTransformer("all-mpnet-base-v2")

# Get embeddings for words
embeddings = model.encode(keywords_df["keywords_list"].tolist(), convert_to_tensor=False)

# Cluster on embeddings
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
keywords_df["cluster"] = kmeans.fit_predict(embeddings)

print(keywords_df.sort_values('keywords_list'))
print(keywords_df["cluster"].value_counts())


# 🔑 Flatten and Cluster Keywords

We are using **SentenceTransformers** to embed the keywords and then applying **KMeans clustering** with `n_clusters = 10`.

Steps performed:
1. Flatten the keywords and create a DataFrame.
2. Generate embeddings using `all-mpnet-base-v2`.
3. Apply KMeans clustering on embeddings.
4. Assign cluster labels to each keyword.
5. Display sorted keywords and cluster counts.


    keywords_list  cluster
4           acads        1
5          action        0
6      activities        8
7     advertising        4
8      aesthetics        7
..            ...      ...
741         years        0
742       youtube        1
743       zenless        3
744          zone        9
745           ˆᗜˆ        9

[749 rows x 2 columns]
cluster
7    118
9     94
1     92
5     82
3     74
0     74
4     74
6     52
2     50
8     39
Name: count, dtype: int64


In [78]:
clustered_df = (
    keywords_df.groupby("cluster")["keywords_list"]
    .apply(list)
    .reset_index()
    .rename(columns={"keywords_list": "words"})
)

clustered_df

,cluster,words
0,0,"[action, anything, appreciation, arrangements,..."
1,1,"[cybersecurity, acads, aftermarket, applicatio..."
2,2,"[archery, badminton, basketball, bball, beginn..."
3,3,"[m, ve, ako, animanga, animo, ayaw, branched, ..."
4,4,"[advertising, baking, binge, bingewatching, bi..."
5,5,"[ai, analysis, analytics, animation, architect..."
6,6,"[video_games, billiards, cello, chess, csgo, c..."
7,7,"[aesthetics, animal, anime, art, articles, art..."
8,8,"[activities, bike, biking, bouldering, camping..."
9,9,"[annoying, assist, atm, baker, based, bass, be..."


In [79]:
n_clusters = kmeans.n_clusters
n_members = len(lscs_sorted)
embedding_dim = kmeans.cluster_centers_.shape[1]

# Encode all members safely
vectors_list = []
for kw in lscs_sorted["keywords_list"]:
    if kw:
        vec = model.encode(kw, convert_to_tensor=False)
        vectors_list.append(np.mean(vec, axis=0) if len(vec) > 0 else np.zeros(embedding_dim))
    else:
        vectors_list.append(np.zeros(embedding_dim))

vectors = np.vstack(vectors_list)

# Compute distances to centroids
distances = np.linalg.norm(vectors[:, None, :] - kmeans.cluster_centers_[None, :, :], axis=2)

# Sort preferences (closest centroid first)
preferences = np.argsort(distances, axis=1)

# Initialize assignment
assigned = np.full(n_members, -1)
cluster_counts = np.zeros(n_clusters, dtype=int)

# Define max allowed difference between largest and smallest cluster
max_diff = 20
ideal_size = n_members // n_clusters

# Greedy assignment with max difference check
for member_idx in range(n_members):
    for pref in preferences[member_idx]:
        if cluster_counts[pref] <= min(cluster_counts) + max_diff:
            assigned[member_idx] = pref
            cluster_counts[pref] += 1
            break
    # fallback: assign to cluster with fewest members if still unassigned
    if assigned[member_idx] == -1:
        min_cluster = np.argmin(cluster_counts)
        assigned[member_idx] = min_cluster
        cluster_counts[min_cluster] += 1

lscs_sorted["assigned_cluster"] = assigned
lscs_sorted["assigned_cluster"]

,assigned_cluster
642,9
5,9
793,7
367,5
742,6
...,...
821,0
628,1
156,3
768,5


In [80]:
def top_keywords_per_cluster(kmeans, embeddings, keywords_df, top_n=5):
    cluster_top_words = {}

    for cluster_id in range(kmeans.n_clusters):
        # Get all members of this cluster
        members_idx = np.where(keywords_df["cluster"] == cluster_id)[0]
        member_vecs = embeddings[members_idx]

        if len(members_idx) == 0:
            cluster_top_words[cluster_id] = []
            continue

        # Compute distances of members to centroid
        dists = np.linalg.norm(member_vecs - kmeans.cluster_centers_[cluster_id], axis=1)

        # Sort members by closeness to centroid
        sorted_idx = members_idx[np.argsort(dists)]

        # Take top_n representative words
        top_words = keywords_df.iloc[sorted_idx[:top_n]]["keywords_list"].tolist()
        cluster_top_words[cluster_id] = top_words

        print(f"\nCluster {cluster_id} top keywords: {', '.join(top_words)}")

    return cluster_top_words

# --- Run function ---
cluster_top_keywords = top_keywords_per_cluster(kmeans, embeddings, keywords_df, top_n=5)


Cluster 0 top keywords: things, sales, stuff, collectibles, pieces

Cluster 1 top keywords: computers, networks, tech, computer, pcs

Cluster 2 top keywords: sports, wrestling, boxing, volleyball, basketball

Cluster 3 top keywords: chunithm, kahit, kdrama, magpaganda, chiikawa

Cluster 4 top keywords: consuming, eating, working, binging, putting

Cluster 5 top keywords: coding, programming, designing, codeing, development

Cluster 6 top keywords: gaming, games, videogames, game, rpgs

Cluster 7 top keywords: literature, storytelling, philosophy, culture, religion

Cluster 8 top keywords: cycling, riding, biking, traveling, hiking

Cluster 9 top keywords: u, played, presented, k, watch


In [81]:
# Merge using House ID instead of Color Category
lscs_with_house = lscs_sorted.merge(
    house_df[["House ID", "House Name"]],
    left_on="assigned_cluster",
    right_on="House ID",
    how="left"
)

# Drop duplicate House ID column
lscs_with_house = lscs_with_house.drop(columns=["House ID"])


#Rename Column
lscs_with_house = lscs_with_house.rename(columns={"House Name": "house_name"})

# Display only the needed columns
lscs_with_house


,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link,cleaned_interests,keywords_list,keywords,assigned_cluster,house_name
0,12505196,"Arous, Adam",NaN,adam_arous@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Information Technology,NaN,many,0976 216 3937,https://www.facebook.com/adam.arousii?mibextid...,many,[],,9,House of Rose Quartz
1,12107884,"Rejano, Hans Martin F.",NaN,hans_rejano@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science,NaN,?,0929 236 6554,fb.com/rejano03,?,[],,9,House of Rose Quartz
2,12512478,"Sergio, Sean Gabriel Steven G.",NaN,sean_sergio@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Science,BS Computer Science Major in Software Technology,NaN,"Basketball, Art, Gaming, Game dev, Language Le...",09695320125,https://www.facebook.com/sean.sergioSS/,"Basketball, Art, Gaming, Game dev, Language Le...","[anime, basketball, art, learning, language, g...","anime, basketball, art, learning, language, ga...",7,House of Moonstone
3,12411272,"Miranda, Bien Aouien C.",NaN,bien_aouien_miranda@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science,NaN,calisthenics\nStudying Japanese\nProgramming\n...,09690488856,https://www.facebook.com/bienaouien.miranda/,"calisthenics,Studying Japanese,Programming,Ani...","[anime, calisthenics, gaming, studying, progra...","anime, calisthenics, gaming, studying, program...",5,House of Lapis Lazuli
4,12506826,"Valmonte, Shaira Vernette D.",NaN,shaira_valmonte@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies,BS - Information Technology,NaN,"Video games, volleyball, dancing, anime, kpop",0960 603 2215,https://www.facebook.com/share/17GiGAoiV3/?mib...,"Video games, volleyball, dancing, anime, kpop","[anime, dancing, kpop, video_games, volleyball]","anime, dancing, kpop, video_games, volleyball",6,House of Amethyst
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,12536334,"Bautista, Naomi Xandrine, L.",NaN,naomi_bautista@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Interactive Entertainment,NaN,"Art & Creativity – I enjoy drawing, digital pa...",0965 677 1951,https://www.facebook.com/share/19fFx3BBXH/?mib...,"Art , Creativity – I enjoy drawing, digital pa...","[years, solitaire, –_i_like, reading, –_i_like...","years, solitaire, –_i_like, reading, –_i_liked...",0,House of Jasper
849,12504920,"Eborde, Mikaelo D.",NaN,elo_eborde@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies,"BS - Computer Science, Major in Software Techn...",NaN,Music Production (with speciality in electroni...,0939 520 1808,https://www.facebook.com/mikaelo.eborde/,Music Production (with speciality in electroni...,"[years, spec, pop, articles, handled, website,...","years, spec, pop, articles, handled, website, ...",1,House of Carnelian
850,12307327,"Llanes, Andre Gabriel",NaN,andre_llanes@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science Major in Software Techno...,NaN,"zenless zone zero, dogs",0956 029 6425,https://www.facebook.com/andregabgd/,"zenless zone zero, dogs","[zone, zenless, dogs]","zone, zenless, dogs",3,House of Aventurine
851,12507261,"Chicombing, Bryce Nathan, S.",NaN,bryce_nathan_chicombing@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science Major in Network and Inf...,NaN,interested in computer and technology 👍,09696104493,https://www.facebook.com/bryce.nathan.52/,interested in computer and technology 👍,"[👍, technology, computer]","👍, technology, computer",5,House of Lapis Lazuli


In [82]:
def top_keywords_per_assigned_house(kmeans, embeddings, keywords_df, assigned_clusters, house_df, top_n=10):
    cluster_top_words = {}
    n_clusters = kmeans.n_clusters

    # Create mapping from cluster ID to house name
    cluster_to_house = dict(zip(house_df["House ID"], house_df["House Name"]))

    for cluster_id in range(n_clusters):
        # Get all members assigned to this cluster
        members_idx = np.where(assigned_clusters == cluster_id)[0]
        member_vecs = embeddings[members_idx]

        if len(members_idx) == 0:
            cluster_top_words[cluster_id] = []
            continue

        # Compute distances of members to the cluster centroid
        dists = np.linalg.norm(member_vecs - kmeans.cluster_centers_[cluster_id], axis=1)

        # Sort members by closeness to centroid
        sorted_idx = members_idx[np.argsort(dists)]

        # Flatten keywords from all members
        all_keywords = []
        for idx in sorted_idx:
            all_keywords.extend(keywords_df.iloc[idx]["keywords_list"])

        # Take top N most common keywords
        top_words = [word for word, _ in Counter(all_keywords).most_common(top_n)]
        cluster_top_words[cluster_id] = top_words

        house_name = cluster_to_house.get(cluster_id, f"House {cluster_id}")
        print(f"\nHouse {house_name} top {top_n} keywords: {', '.join(top_words)}")

    return cluster_top_words

# --- Run function ---
cluster_top_keywords_mapped = top_keywords_per_assigned_house(
    kmeans, vectors, lscs_sorted, lscs_sorted["assigned_cluster"].values, house_df, top_n=10
)


House House of Jasper top 10 keywords: nan, games, music, video_games, watching, gaming, reading, playing, listening, sports

House House of Carnelian top 10 keywords: nan, music, gaming, drawing, video_games, reading, computers, programming, watching, coding

House House of Citrine top 10 keywords: nan, basketball, sports, gaming, gym, games, playing, video_games, music, running

House House of Aventurine top 10 keywords: nan, programming, reading, music, manga, gaming, anime, sports, manhwa, watching_movies

House House of Amazonite top 10 keywords: nan, reading, gaming, cooking, playing, coding, watching, programming, music, running

House House of Lapis Lazuli top 10 keywords: coding, nan, programming, gaming, video_games, music, photography, computer, art, drawing

House House of Amethyst top 10 keywords: nan, music, video_games, gaming, badminton, games, programming, guitar, playing, volleyball

House House of Moonstone top 10 keywords: music, nan, reading, movies, gaming, games

In [83]:
# Remove Features and Just Display the Housee Names
lscs_with_house = lscs_with_house.drop(columns=['cleaned_interests', 'keywords_list', 'keywords','assigned_cluster'])


In [84]:
lscs_with_house

,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link,house_name
0,12505196,"Arous, Adam",NaN,adam_arous@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Information Technology,NaN,many,0976 216 3937,https://www.facebook.com/adam.arousii?mibextid...,House of Rose Quartz
1,12107884,"Rejano, Hans Martin F.",NaN,hans_rejano@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science,NaN,?,0929 236 6554,fb.com/rejano03,House of Rose Quartz
2,12512478,"Sergio, Sean Gabriel Steven G.",NaN,sean_sergio@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Science,BS Computer Science Major in Software Technology,NaN,"Basketball, Art, Gaming, Game dev, Language Le...",09695320125,https://www.facebook.com/sean.sergioSS/,House of Moonstone
3,12411272,"Miranda, Bien Aouien C.",NaN,bien_aouien_miranda@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science,NaN,calisthenics\nStudying Japanese\nProgramming\n...,09690488856,https://www.facebook.com/bienaouien.miranda/,House of Lapis Lazuli
4,12506826,"Valmonte, Shaira Vernette D.",NaN,shaira_valmonte@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies,BS - Information Technology,NaN,"Video games, volleyball, dancing, anime, kpop",0960 603 2215,https://www.facebook.com/share/17GiGAoiV3/?mib...,House of Amethyst
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,12536334,"Bautista, Naomi Xandrine, L.",NaN,naomi_bautista@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Interactive Entertainment,NaN,"Art & Creativity – I enjoy drawing, digital pa...",0965 677 1951,https://www.facebook.com/share/19fFx3BBXH/?mib...,House of Jasper
849,12504920,"Eborde, Mikaelo D.",NaN,elo_eborde@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies,"BS - Computer Science, Major in Software Techn...",NaN,Music Production (with speciality in electroni...,0939 520 1808,https://www.facebook.com/mikaelo.eborde/,House of Carnelian
850,12307327,"Llanes, Andre Gabriel",NaN,andre_llanes@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science Major in Software Techno...,NaN,"zenless zone zero, dogs",0956 029 6425,https://www.facebook.com/andregabgd/,House of Aventurine
851,12507261,"Chicombing, Bryce Nathan, S.",NaN,bryce_nathan_chicombing@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science Major in Network and Inf...,NaN,interested in computer and technology 👍,09696104493,https://www.facebook.com/bryce.nathan.52/,House of Lapis Lazuli


In [88]:
lscs_with_house["house_name"].value_counts()

,count
house_name,
House of Moonstone,89
House of Lapis Lazuli,89
House of Jasper,89
House of Amethyst,89
House of Carnelian,89
House of Amazonite,89
House of Rose Quartz,88
House of Citrine,88
House of Aventurine,75


In [85]:
lscs_with_house.sort_values('house_name').to_csv("lscs_with_house.csv", index=False)